In [1]:
# A simple DTW Notebook for string matching and error rate evaluation

In [2]:
# do all the imports
%matplotlib inline

import sys, os
import numpy as np
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import time

In [3]:
# graphical and print preferences
ldesign = 50
cmap = sns.light_palette("caramel",ldesign,input="xkcd")
cmap20 = cmap[0:20] 
pd.reset_option('display.float_format')
pd.set_option('precision',3)

In [64]:
def dtw(obs="",ref="",iC=1.,dC=1.,sC=1.,print_Trellis=False,print_Col=False):
    ''' 
    Dynamic Time Warping
        finds the best alignment between two strings (to be parsed on blanks)
        using Levensthein Minimal Edit Distance paradigm
        with flexible insertion, deletion and substitution costs
        
    obs = observation string   (atoms will be found after parsing for white space)
    ref = reference string     (atoms will be found after parsing for white space)
    iC, dC, sC    insertion, deletion and substitution costs (default=1.0)
    
    '''
    # add dummy symbol in front to accomodate initial insertions and deletions
    x = ["/"]
    x.extend(obs.split())
    y = ["/"]
    y.extend(ref.split())

    # initialize data structures, for easy backtracking we remember backpointers AND type of edit
    # trellis = matrix of cummulative distances  (len(y),len(x)) float's
    # bptr    = tensor of backpointers           (len(y),len(x),2) int's
    # edits   = matrix of edits                  (len(y),len(x)) U1
    trellis = np.zeros((len(y),len(x)))
    bptr = np.zeros((len(y),len(x),2),dtype='int')
    edits = np.full((len(y),len(x)),"M",dtype='str')
    
    # initialize full row of deletions and insertions
    for ix in range(1,len(x)): 
        trellis[0,ix] = trellis[0,ix-1] + iC
        bptr[0,ix] = [0,ix-1]
        edits[0,ix]='I'
    for iy in range(1,len(y)): 
        trellis[iy,0] = trellis[iy-1,0] + dC
        bptr[iy,0] = [iy-1,0]
        edits[iy,0]='D'
        
    # compute accumulated distance with standard trellis recursions
    for ix in range(1,len(x)):
        
        for iy in range(1,len(y)):
            is_sub = (x[ix] != y[iy])
            sScore = trellis[iy-1,ix-1] + sC * int(is_sub)
            iScore = trellis[iy,ix-1] + iC
            dScore = trellis[iy-1,ix] + dC
            
            if( (sScore < iScore) & (sScore < dScore) ):  # substitution or match
                trellis[iy,ix] = sScore
                bptr[iy,ix] = [iy-1,ix-1]
                if is_sub: edits[iy,ix] = "S"
                else: edits[iy,ix]="M"
            elif( iScore < dScore ):   
                trellis[iy,ix] = iScore
                bptr[iy,ix] = [iy,ix-1]
                edits[iy,ix] = "I" 
            else:
                trellis[iy,ix] = dScore
                bptr[iy,ix] = [iy-1,ix]
                edits[iy,ix] = "D" 
                
        if(print_Col):
            print(pd.DataFrame(trellis,index=y,columns=x))
            time.sleep(5)
            
    if(print_Trellis):        
        print(pd.DataFrame(trellis,index=y,columns=x))
        print(pd.DataFrame(edits,index=y,columns=x))

# find the alignment via backtracking over the backpointers, omit [0,0] at the end
    ix = len(x)-1
    iy = len(y)-1
    btr = [iy,ix]
    (iy,ix) = bptr[iy,ix]
    while((ix > 0) & (iy >0)):
        btr = np.vstack([[iy, ix],btr])
        (iy,ix) = bptr[iy,ix]
        
    alignment = []
    editstring = []
    xalign=[]
    yalign=[]
    editseq=[]
    for i in range(0,len(btr)):
        iy= btr[i,0]
        ix =btr[i,1]
        ystr = y[btr[i,0]]
        xstr = x[btr[i,1]]
        if(edits[iy,ix] == 'D' ): xstr = '-'
        if(edits[iy,ix] == 'I' ): ystr = '-'
        xalign.append(xstr)
        yalign.append(ystr)
        editseq.append(edits[iy,ix])

    nsub = sum(editseq[i]=='S' for i in range(0,len(editseq)))
    nins = sum(editseq[i]=='I' for i in range(0,len(editseq)))    
    ndel = sum(editseq[i]=='D' for i in range(0,len(editseq)))
    # don't count special symbols <.... > in word length
    # ... is not properly taken care of in counting subs/ins/del !!

    y2 = [ yy for yy in y if not re.match(re.compile('<.*>'),yy) ] 
    nwords = len( y2 )
    
    display(pd.DataFrame({"reference":yalign,"observation":xalign,"EDIT":editseq}))
    print("#SUB: %d, #INS: %d, #DEL: %d, #W(REF): %d,  WER=%.2f%%" % 
          (nsub,nins,ndel,nwords,100.*float(nsub+nins+ndel)/float(nwords) ))

In [65]:
ins_cost = 1.
del_cost = 1.
sub_cost = 1.
observation = "d o n y s o   s"
reference = "d o t e o s "
dtw(obs=observation, ref=reference, iC=ins_cost,dC=del_cost,sC=sub_cost,print_Trellis=False)

,reference,observation,EDIT
0,d,d,M
1,o,o,M
2,t,n,S
3,e,y,S
4,-,s,I
5,o,o,M
6,s,s,M


#SUB: 2, #INS: 1, #DEL: 0, #W(REF): 7,  WER=42.86%


In [66]:
ref1="fauchelevent limped along behind the horse in a very contented frame of mind "
obs1 = "lochleven limped along behind the heard in very contented frame of mind"
dtw(obs=obs1, ref=ref1)
ref2= " he would have loved to be king in such a non nonsense paradise "
obs2=" he had loved the king in a no sense paradigm"
dtw(obs=obs2, ref=ref2)
ref3 = "do you know the names of the seven dwarfs in Disney’s Snow White movie ?"
obs3 =  "do you know the names of the seven warfs in the sneaze now white movie ?"
dtw(obs=obs3, ref=ref3)

,reference,observation,EDIT
0,fauchelevent,lochleven,S
1,limped,limped,M
2,along,along,M
3,behind,behind,M
4,the,the,M
5,horse,heard,S
6,in,in,M
7,a,-,D
8,very,very,M
9,contented,contented,M


#SUB: 2, #INS: 0, #DEL: 1, #W(REF): 14,  WER=21.43%


,reference,observation,EDIT
0,he,he,M
1,would,had,S
2,have,-,D
3,loved,loved,M
4,to,the,S
5,be,-,D
6,king,king,M
7,in,in,M
8,such,-,D
9,a,a,M


#SUB: 5, #INS: 0, #DEL: 3, #W(REF): 14,  WER=57.14%


,reference,observation,EDIT
0,do,do,M
1,you,you,M
2,know,know,M
3,the,the,M
4,names,names,M
5,of,of,M
6,the,the,M
7,seven,seven,M
8,dwarfs,warfs,S
9,in,in,M


#SUB: 4, #INS: 1, #DEL: 0, #W(REF): 16,  WER=31.25%


In [69]:
# we can concatenate to one long string assuming that misalignment will not occur
# across sentence boundaries
# make sure that there is white space between the sentences
# alternatively you could enforce boundary alignment by adding specific </s> symbols
obs = obs1 + "  " + obs2 + " " + obs3
ref = ref1 + " </s> " + ref2 + " " + ref3
dtw(obs=obs, ref=ref)

,reference,observation,EDIT
0,fauchelevent,lochleven,S
1,limped,limped,M
2,along,along,M
3,behind,behind,M
4,the,the,M
5,horse,heard,S
6,in,in,M
7,a,-,D
8,very,very,M
9,contented,contented,M


#SUB: 11, #INS: 1, #DEL: 5, #W(REF): 42,  WER=40.48%
